In [ ]:
import requests
import time

BASE_URL = "https://korean.visitkorea.or.kr/api/v2/hot-place/place/list"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Referer": "https://korean.visitkorea.or.kr/",
}

params = {
    "page": 1,
    "offset": 15,
    "device": "PC",
    "hotPlaceType": "Place",
    "order": "POPULAR",
    "latitude": 0,
    "longitude": 0,
}

def make_image_url(image_id):
    if not image_id:
        return None
    return f"https://cdn.visitkorea.or.kr/img/call?cmd=VIEW&id={image_id}"

seen_ids = set()
results = []

while True:
    res = requests.get(BASE_URL, headers=headers, params=params)
    data = res.json()

    items = data["data"]["items"]
    total = data["data"]["total"]

    if not items:
        break

    for item in items:
        coid = item["cotId"]

        # ✅ 중복 방지
        if coid in seen_ids:
            continue

        seen_ids.add(coid)

        results.append({
            "coid": coid,
            "name": item["title"],
            "addr": item["detailDatabase"].get("addr1", ""),
            "image_url": make_image_url(item["detailDatabase"].get("firstImage")),
            "overview": item["detailDatabase"].get("overView", "")
        })

    print(f"{params['page']}페이지 수집 완료 ({len(items)}개)")

    # ✅ 종료 조건 (서버 기준)
    if params["page"] * params["offset"] >= total:
        break

    params["page"] += 1
    time.sleep(1.3)

print("총 수집 개수:", len(results))


In [ ]:
import csv

CSV_PATH = "visitkorea_places.csv"

with open(CSV_PATH, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(
        f,
        fieldnames=["coid", "name", "addr", "image_url", "overview"]
    )
    writer.writeheader()
    writer.writerows(results)

print("CSV 저장 완료:", CSV_PATH)
